In [2]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3566 sha256=ef9b6dc2ae51bdb6a0e08112500b89ec0d4dc9957708c7c21ba27c876f71fbd1
  Stored in directory: /home/ebezerra/.cache/pip/wheels/b2/b2/20/c0960e8ee3ceaf158c43f28eea50357113dfe2f3106da9fdb1
Successfully built fastdtw


See https://youtu.be/X6phfLqN5pY

In [3]:
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# Generate two example time series with different temporal resolutions
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) # Hourly resolution
b = np.array([1, 3, 6, 9]) # Observed every 12 hours

# Define the distance function to use for DTW
def dist(x, y):
    return euclidean(x, y)

# Use fastdtw to calculate the DTW path and total distance
distance, path = fastdtw(a.reshape(-1,1), b.reshape(-1,1), dist=dist)

# Print the DTW path and total distance
print("DTW path:", path)
print("Total distance:", distance)

# Align the two time series using the DTW path
aligned_a = np.zeros_like(b)
for i, j in path:
    aligned_a[j//2] = a[i]

# Print the aligned time series
print("Aligned a:", aligned_a)
print("b:", b)

DTW path: [(0, 0), (1, 1), (2, 1), (3, 1), (4, 2), (5, 2), (6, 2), (7, 3), (8, 3), (9, 3)]
Total distance: 6.0
Aligned a: [ 4 10  0  0]
b: [1 3 6 9]


In [6]:
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

# Define two time series with different temporal resolutions
# One series has hourly temporal resolution, and the other has 12-hourly temporal resolution
hourly_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
twelve_hour_data = np.array([11, 13, 14, 16])

# Define a function to calculate the distance between two data points
def distance(x, y):
    return euclidean(x, y)

# Use fastdtw to calculate the DTW distance and alignment path between the two time series
distance, path = fastdtw(hourly_data.reshape(-1,1), twelve_hour_data.reshape(-1,1), dist=distance)

# Create a new time series with hourly temporal resolution that is aligned with the original hourly data
aligned_data = np.zeros_like(hourly_data)
last_index = 0
for i in range(len(hourly_data)):
    if path[i][1] > last_index:
        last_index = path[i][1]
    aligned_data[i] = twelve_hour_data[last_index]

print("Original hourly data:", hourly_data)
print("Original 12-hourly data:", twelve_hour_data)
print("Aligned data with hourly temporal resolution:", aligned_data)


Original hourly data: [ 1  2  3  4  5  6  7  8  9 10]
Original 12-hourly data: [11 13 14 16]
Aligned data with hourly temporal resolution: [11 11 11 11 11 11 11 13 14 16]


The following is an example of PyTorch code to fuse data from sensors with different temporal resolutions using a recurrent neural network:

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define the RNN model
class SensorFusionRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SensorFusionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Define the training dataset
class SensorDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Set the device to run on (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Generate some sample data
n_samples = 1000
hourly_data = torch.randn(n_samples, 24, 3) # Three hourly time series
daily_data = torch.randn(n_samples, 2, 5) # Two daily time series
labels = torch.randint(0, 2, (n_samples,))

# Resample the daily data to hourly resolution
daily_data = daily_data.repeat_interleave(12, dim=1)

# Concatenate the two types of data
X = torch.cat([hourly_data, daily_data], dim=2)

# Split the data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = labels[:train_size], labels[train_size:]

# Create data loaders for the training and testing sets
batch_size = 32
train_dataset = SensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = SensorDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
input_size = hourly_data.shape[2] + daily_data.shape[2]
hidden_size = 64
num_layers = 2
output_size = 2
model = SensorFusionRNN(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
n_epochs = 10
for epoch in range(n_epochs):
    train_loss = 0.0
    train_correct = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_X
    print(f"Epoch: {epoch}. Train loss: {train_loss.mean()}")


Epoch: 0. Train loss: 0.053246837109327316
Epoch: 1. Train loss: 0.05263359099626541
Epoch: 2. Train loss: 0.054822277277708054
Epoch: 3. Train loss: 0.05240136384963989
Epoch: 4. Train loss: 0.05091685801744461
Epoch: 5. Train loss: 0.05948447808623314
Epoch: 6. Train loss: 0.05005837604403496
Epoch: 7. Train loss: 0.05503062531352043
Epoch: 8. Train loss: 0.048353999853134155
Epoch: 9. Train loss: 0.0538087859749794


In [18]:
n_samples = 4 # number of days
hourly_data = torch.randn(n_samples, 24, 3) # Three hourly time series
daily_data = torch.randn(n_samples, 2, 5) # Two daily time series
labels = torch.randint(0, 2, (n_samples,))

In [16]:
hourly_data

tensor([[[-1.8172e+00, -1.3488e+00, -8.1698e-01],
         [-1.3653e-01, -2.2581e-01,  5.3250e-01],
         [ 4.0494e-01, -1.0501e+00,  1.3792e-01],
         [-1.0741e+00,  1.2342e+00, -2.5167e+00],
         [-1.4267e+00,  1.8778e-01, -5.5419e-01],
         [ 8.0680e-01,  1.2124e-01, -1.5800e-01],
         [ 5.9127e-01, -1.0306e+00,  4.2683e-01],
         [ 1.5380e+00, -3.3456e-01,  7.3039e-01],
         [-2.7381e+00,  1.2795e+00,  2.7452e-01],
         [ 1.7857e+00, -1.2792e+00,  3.5183e-01],
         [ 3.2702e-01,  3.5998e-01,  6.3462e-01],
         [ 1.3335e+00,  1.1580e+00,  1.5274e-01],
         [-2.4153e+00,  6.5580e-01, -1.2506e+00],
         [ 9.2260e-01,  2.5447e-02,  9.1569e-01],
         [-1.2832e-01,  3.3937e-01, -2.9902e-01],
         [ 1.1201e-01, -5.3755e-01,  7.5297e-01],
         [ 1.0984e+00, -1.4439e+00,  7.8407e-01],
         [-1.0973e-01,  8.5727e-01, -6.2663e-01],
         [-4.0856e-01,  2.1308e-01, -1.3866e-01],
         [-1.7072e+00,  4.8194e-01, -2.9714e-02],


In [17]:
daily_data

tensor([[[-0.4531,  0.6991,  0.0212, -0.4505, -0.9256],
         [-1.0419, -0.2938, -0.7084,  0.5654, -0.6895]],

        [[-1.0420, -2.0822,  0.1945,  1.2655, -0.2160],
         [-0.4643,  0.3460, -0.2672, -0.0413, -0.9296]],

        [[-0.9134,  0.4023, -1.3692,  0.7514, -1.8142],
         [-0.0551, -1.6823,  0.8246, -0.7286,  1.5110]],

        [[ 0.6358,  1.3754, -1.0702, -1.7944,  1.7589],
         [-2.3769, -0.4385,  1.9920, -0.1218,  0.4564]]])

In [20]:
# Resample the daily data to hourly resolution
daily_data = daily_data.repeat_interleave(12, dim=1)

In [21]:
daily_data

tensor([[[-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [-0.4762, -0.4869, -0.5233,  0.9426,  1.8478],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2.2831],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2.2831],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2.2831],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2.2831],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2.2831],
         [ 0.1830,  0.2630, -0.6110,  0.9658,  2

In [22]:
X = torch.cat([hourly_data, daily_data], dim=2)
print(X)

tensor([[[ 5.0435e-02,  6.2132e-02,  4.7415e-02, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 1.8389e+00, -3.5803e-01, -4.4248e-01, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 2.6162e-01,  5.6249e-01, -4.1762e-01, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 1.7359e+00,  5.8134e-01,  4.7416e-01, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 1.0735e+00,  5.9101e-02, -1.7606e-02, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 3.5185e-01,  1.8197e-02,  1.6766e+00, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [-5.6444e-01, -1.8336e-01, -1.2404e+00, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],
         [ 7.4576e-02,  1.1095e+00, -2.3928e-01, -4.7618e-01, -4.8686e-01,
          -5.2333e-01,  9.4260e-01,  1.8478e+00],


In [23]:
labels = torch.randint(0, 2, (n_samples,))
print(labels)

tensor([0, 0, 1, 1])
